# Data Acquisition Example: Testing WLAN Client Receiver System Noise

Test data for WLAN is implemented with the the iperf support in [ssmdevices](https://gitlab.nist.gov/ssm/ssmdevices). It produces many more columns of data than we need, so we just use the two defined in `ipc_columns`: throughput and a timestamp.

The results of the tests and the corresponding test conditions are stored in a [flat database](https://en.wikipedia.org/wiki/Flat_file_database) stored in SQLite format. It is implemented as a typical intended use case of `labbench.RelationalDataInSQLite`. The test conditions are implemented as states in the attenuators, `iperf`, and `wlan`, so all we need to log these results into the database is to add the `db.on_set` call. Any states that are changed after that function call (for example, with the for loops in the acquisition code) become columns in the database; these values are automatically kept up to date and written to the database on calls to `db.write`.

In [1]:
# This should be run in python 3.6.x from the computer connected to the AP side

%pylab inline
import time,os
import ssmdevices as ssm
import labbench as lb
import pandas as pd

def meshpoints (x1, x2, *args):
    ''' For input of N-dimensions of grid point vectors (x1, x2, ..., xN),
        where each (x1, x2, ... xN) has dimension (M1, M2, ..., MN),
        returns an array with shape (M1*M2*...*MN,N) that consists of the
        unique points in the multi-dimensional grid with sampled at
        the vector points (x1, x2, ..., xN) in each axis. This is also known
        as the the cartesian product of (x1 ... xN).
    '''
    ret = np.array(np.meshgrid(*((x1,x2)+args)))
    return ret.reshape([ret.shape[0],np.prod(ret.shape[1:])]).T

setup_time = 5 # seconds
acquisition_time = 1 #  seconds

# iperf_columns       = ['iperf_bits_per_second','iperf_timestamp']
sweep_points       = meshpoints(list(range(55,76,20)),[110]+list(range(0,21,20)))

# db_path = '{}'.format(time.strftime("%Y-%m-%d-%HH-%MM"))

lb.show_messages('info')


def update(update):
    states[update['name']] = update['new']

import time

for i in range(1):
    t0 = time.time()
    states = {}
    
    np.random.shuffle(sweep_points) # randomize?
    
    c = ssm.instruments.MiniCircuitsRCDAT('11604210008')
    e0 = ssm.instruments.MiniCircuitsRCDAT('11604210014')
    iperf_client = ssm.software.IPerf('10.0.0.2', bind='10.0.0.3', port=5010, interval=0.1)
    iperf_server = ssm.software.IPerf(bind='10.0.0.2', port=5010)
    wlan = ssm.software.WLANStatus('Wi-Fi', ssid='EnGenius1')
    db = lb.StatesToSQLite(time.strftime("%Y-%m-%d-%H%M%S with shuffle and absorber"))
    
    # Test progress heads-up display
    lb.panel()
    
    # This `with' block ensures sockets and iperf subprocesses are
    # closed correctly when the script ends, even if exceptions are raised
    with lb.concurrently(c, e0, iperf_client, iperf_server, wlan, db):
        try:
            # Log all state changes in lte_laa and each client in iperf_clients
            db.observe_states([c,e0,iperf_server,iperf_client,wlan])

            # Only need to do this if there is a variable attenuator in the test setup
            # lte_enb_atten.state.attenuation = 20. # dB

            wlan.interface_disconnect()
            c.state.attenuation = 30
            e0.state.attenuation = 110

            iperf_server.start()
            iperf_client.start()

            lb.logger.info('Pausing for {}s setup time'.format(setup_time))
            lb.sleep(setup_time)

            df=iperf_client.fetch()
            df_server=iperf_server.fetch()

            iperf_server.kill()
            iperf_client.kill()

            row = {}
            i = 0

            for c.state.attenuation, e0.state.attenuation in lb.log_progress(sweep_points):
                p = iperf_server.settings.port
                iperf_client.settings.port = iperf_server.settings.port = p+1            

                i += 1
                lb.logger.debug('{}/{}: atten C {} dB, atten E0 {} dB'\
                               .format(i, len(sweep_points), c.state.attenuation, e0.state.attenuation))

                try:
                    wlan.interface_disconnect(5)
                except TimeoutError as e:
                    lb.logger.debug(str(e))
                    lb.logger.debug('Failed to disconnect from AP, continuing anyway')

                try:
                    # Connect
                    wlan.interface_connect(5)
                except TimeoutError as e:
                    lb.logger.debug(str(e))
                    lb.logger.debug('No connection to AP - skipping iperf')
                    df = None                
                else:
                    # Acquire
                    iperf_server.start()
                    iperf_client.start()

                    lb.sleep(acquisition_time)

                    df = iperf_client.fetch()
                    df_server = iperf_server.fetch()

                    lb.logger.debug('  iperf_client and server returned {} and {} rows'\
                                   .format(len(df),len(df_server)))

                    iperf_server.kill()
                    iperf_client.kill()               
                finally:
                    # Update all WLAN states before append 
                    wlan.refresh()

                    db.append(iperf_client=df)
                    db.write()
        finally:
            iperf_server.kill()
            iperf_client.kill()

    # # # Load the sqlite database and save a copy into a csv
    df = lb.read(os.path.join(db.path, 'master.db'))
    df.to_csv(os.path.join(db.path,'master.csv'))

Populating the interactive namespace from numpy and matplotlib


C:\Users\dkuester\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [ ]:
np.random.shuffle(sweep_points)

In [ ]:
df = lb.read(r'C:/Users/dkuester/Documents/src/ssmdevices/examples/2018-10-30-141739 with shuffle and absorber/master.db')
df2 = lb.read(r'C:\Users\dkuester\Documents\src\ssmdevices\examples\2018-10-30-154819 with shuffle and absorber\master.db')

df=df.sort_values(['c_attenuation','e0_attenuation'])
df2=df2.sort_values(['c_attenuation','e0_attenuation'])
df=df.set_index(['c_attenuation','e0_attenuation'])
df2=df2.set_index(['c_attenuation','e0_attenuation'])

# i = (df['wlan_state']=='connected')&(df2['wlan_state']=='connected')

(df['wlan_signal']-df2['wlan_signal']).hist(bins=20)
print((df[i]['wlan_signal']-df2[i]['wlan_signal']).mad())

In [ ]:
df2

In [ ]:
df2.sort_values(['c_attenuation','e0_attenuation'])